In [1]:
#импортируем библиотеки перед началом проекта одним блоком
import pandas as pd

# загружаем классы для подготовки данных
from sklearn.model_selection import train_test_split

# импортируем классы GridSearchCV
from sklearn.model_selection import GridSearchCV

# загружаем нужные модели
from catboost import CatBoostClassifier

# загружаем функцию для работы с метриками
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings('ignore',category=UserWarning)

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
#загрузим данные
orders = pd.read_csv('/Users/roman_yakovlev/Downloads/DataSecrets_DODO/Data_Secrets_First_Cup/orders.csv')
mobile_events = pd.read_csv('/Users/roman_yakovlev/Downloads/DataSecrets_DODO/Data_Secrets_First_Cup/mobile_events.csv')
clients_promo_october = pd.read_csv('/Users/roman_yakovlev/Downloads/DataSecrets_DODO/Data_Secrets_First_Cup/clients_promo_october.csv')
train_target = pd.read_csv('/Users/roman_yakovlev/Downloads/DataSecrets_DODO/Data_Secrets_First_Cup/train_target.csv')
test = pd.read_csv('/Users/roman_yakovlev/Downloads/DataSecrets_DODO/Data_Secrets_First_Cup/test.csv')

In [4]:
# cписок таблиц
tables = [orders, mobile_events, clients_promo_october, train_target, test]

# понижение регистра в названиях столбцов для каждой таблицы
for table in tables:
    table.columns = table.columns.str.lower()

In [5]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611739 entries, 0 to 611738
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   orderuuid          611739 non-null  object 
 1   addressid          488325 non-null  float64
 2   deliverysectorid   488325 non-null  float64
 3   clientuuid         611739 non-null  object 
 4   date               611739 non-null  object 
 5   saledate           611739 non-null  object 
 6   unituuid           611739 non-null  object 
 7   newclient          611739 non-null  int64  
 8   clientordernumber  611739 non-null  int64  
 9   productuuid        611739 non-null  object 
 10  categoryid         611739 non-null  int64  
 11  producttotalprice  611739 non-null  float64
 12  menuprice          611739 non-null  float64
 13  orderstate         611739 non-null  int64  
 14  orderpaymenttype   611739 non-null  int64  
 15  ordertotalprice    611739 non-null  float64
 16  or

In [6]:
# забьем нулями пропуски в кодах адресов и секторов доставки
orders['deliverysectorid'] = orders['deliverysectorid'].fillna(0)
orders['addressid'] = orders['addressid'].fillna(0)

In [7]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611739 entries, 0 to 611738
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   orderuuid          611739 non-null  object 
 1   addressid          611739 non-null  float64
 2   deliverysectorid   611739 non-null  float64
 3   clientuuid         611739 non-null  object 
 4   date               611739 non-null  object 
 5   saledate           611739 non-null  object 
 6   unituuid           611739 non-null  object 
 7   newclient          611739 non-null  int64  
 8   clientordernumber  611739 non-null  int64  
 9   productuuid        611739 non-null  object 
 10  categoryid         611739 non-null  int64  
 11  producttotalprice  611739 non-null  float64
 12  menuprice          611739 non-null  float64
 13  orderstate         611739 non-null  int64  
 14  orderpaymenttype   611739 non-null  int64  
 15  ordertotalprice    611739 non-null  float64
 16  or

In [8]:
mobile_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3916654 entries, 0 to 3916653
Data columns (total 5 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   clientuuid  object
 1   visittoken  object
 2   eventname   object
 3   platform    object
 4   timestamp   object
dtypes: object(5)
memory usage: 149.4+ MB


In [9]:
clients_promo_october.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477275 entries, 0 to 477274
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   clientuuid      477275 non-null  object
 1   id              477275 non-null  int64 
 2   localbegindate  477275 non-null  object
 3   localenddate    477275 non-null  object
 4   ordertype       477275 non-null  object
 5   orderprice      477275 non-null  int64 
 6   discount        477275 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 25.5+ MB


In [10]:
train_target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27741 entries, 0 to 27740
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clientuuid      27741 non-null  object
 1   id              27741 non-null  int64 
 2   ordertype       27741 non-null  object
 3   localbegindate  27741 non-null  object
 4   localenddate    27741 non-null  object
 5   orderprice      27741 non-null  int64 
 6   discount        27741 non-null  int64 
 7   apply_promo     27741 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 1.7+ MB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6806 entries, 0 to 6805
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   clientuuid      6806 non-null   object
 1   id              6806 non-null   int64 
 2   ordertype       6806 non-null   object
 3   localbegindate  6806 non-null   object
 4   localenddate    6806 non-null   object
 5   orderprice      6806 non-null   int64 
 6   discount        6806 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 372.3+ KB


In [12]:
# приведем даты к корреткному типу и посмотрим на уникальные значения дат
train_target['localbegindate']= pd.to_datetime(train_target['localbegindate'])
train_target['localenddate']= pd.to_datetime(train_target['localenddate'])

test['localbegindate']= pd.to_datetime(test['localbegindate'])
test['localenddate']= pd.to_datetime(test['localenddate'])

display(train_target['localbegindate'].unique())
display(train_target['localenddate'].unique())
display(test['localbegindate'].unique())
display(test['localenddate'].unique())

display(clients_promo_october['localbegindate'].unique())
display(clients_promo_october['localenddate'].unique())

<DatetimeArray>
['2023-11-02 00:00:00+00:00']
Length: 1, dtype: datetime64[ns, UTC]

<DatetimeArray>
['2023-11-05 23:59:00+00:00']
Length: 1, dtype: datetime64[ns, UTC]

<DatetimeArray>
['2023-11-02 00:00:00+00:00']
Length: 1, dtype: datetime64[ns, UTC]

<DatetimeArray>
['2023-11-05 23:59:00+00:00']
Length: 1, dtype: datetime64[ns, UTC]

array(['2023-10-12T00:00:00.000Z', '2023-10-19T00:00:00.000Z',
       '2023-10-09T00:00:00.000Z', '2023-10-02T00:00:00.000Z',
       '2023-10-05T00:00:00.000Z', '2023-10-26T00:00:00.000Z',
       '2023-10-30T00:00:00.000Z', '2023-10-16T00:00:00.000Z',
       '2023-10-23T00:00:00.000Z'], dtype=object)

array(['2023-10-15T23:59:00.000Z', '2023-10-22T23:59:00.000Z',
       '2023-10-11T23:59:00.000Z', '2023-10-04T23:59:00.000Z',
       '2023-10-08T23:59:00.000Z', '2023-10-29T23:59:00.000Z',
       '2023-11-01T23:59:00.000Z', '2023-10-18T23:59:00.000Z',
       '2023-10-25T23:59:00.000Z'], dtype=object)

In [13]:
orders

,orderuuid,addressid,deliverysectorid,clientuuid,date,saledate,unituuid,newclient,clientordernumber,productuuid,categoryid,producttotalprice,menuprice,orderstate,orderpaymenttype,ordertotalprice,ordertype,apply_promo
0,020DD76CCD949AE011EDE031ED5D76FD,0.0,0.0,000D3AAC9DCABB2E11EBE0614C89D044,2023-04-21,2023-04-21T20:51:26.000Z,000D3A2480C380DA11E692CFB9F92484,0,18,000D3A240C71BE9A11E719BE2AB264A6,4,1.0,45.0,4,2,1.0,3,0
1,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EC9AABAD354BE051CCF1EE1F83A8D0,1,485.0,485.0,4,2,1164.0,1,0
2,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,11EB846583CB3BDEA80D1A7F2E184D40,1,589.0,589.0,4,2,1164.0,1,0
3,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB267FD,4,45.0,45.0,4,2,1164.0,1,0
4,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02T21:31:32.000Z,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB264A6,4,45.0,45.0,4,2,1164.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611734,FEE4A40E62DA936011EE0DA9C66D87C8,0.0,0.0,000D3A39D824A82E11EA2BB7F48841A0,2023-06-18,2023-06-18T15:34:13.000Z,000D3A22FA54A81411E95B9E966DC9B2,0,55,11EA5EE49DCBD9124A66F764872C8E20,1,594.0,719.0,4,2,1509.0,1,0
611735,FEE4A40E62DA936011EE0DA9C66D87C8,0.0,0.0,000D3A39D824A82E11EA2BB7F48841A0,2023-06-18,2023-06-18T15:34:13.000Z,000D3A22FA54A81411E95B9E966DC9B2,0,55,000D3A39D824A82E11E9A18231161F87,1,395.0,479.0,4,2,1509.0,1,0
611736,FEE7759E6F988C7111EDC1BDEEDF6376,9723140.0,4103.0,000D3A22FA54A81411E9BC5EF27EE5BF,2023-03-24,2023-03-24T15:50:51.000Z,000D3A24D2B7A94311E8706B7529EBFD,0,28,000D3A240C71BE9A11E719BE2AB2D427,3,169.0,169.0,4,2,557.0,1,0
611737,FEE7759E6F988C7111EDC1BDEEDF6376,9723140.0,4103.0,000D3A22FA54A81411E9BC5EF27EE5BF,2023-03-24,2023-03-24T15:50:51.000Z,000D3A24D2B7A94311E8706B7529EBFD,0,28,11ED08AA93691DC615EE94EB19CD7070,3,189.0,189.0,4,2,557.0,1,0


In [14]:
train_target

,clientuuid,id,ordertype,localbegindate,localenddate,orderprice,discount,apply_promo
0,000D3A20F23EA95811E7B99F3ED09FC8,7,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,699,200,0
1,000D3A20F23EA95811E7B99F3ED09FC8,6,"1,2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,1249,25,0
2,000D3A20F23EA95811E7BD373E79565E,5,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,799,200,0
3,000D3A20F23EA95811E7BD373E79565E,5,"1,2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,1249,250,0
4,000D3A20F23EA95811E7BDAE71CD3F33,6,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,699,30,0
...,...,...,...,...,...,...,...,...
27736,82EDBCD6CE60978F11EDC591DCA75401,5,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,799,200,0
27737,82EDBCD6CE60978F11EDC591DCA75401,5,"1,2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,1249,250,0
27738,82F78FC7401A9ADD11EDFA15A350D31F,6,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,799,20,0
27739,8606F5E4CA39BF7211EE517E6E1D90B7,5,"2,3",2023-11-02 00:00:00+00:00,2023-11-05 23:59:00+00:00,749,200,0


In [15]:
clients_promo_october

,clientuuid,id,localbegindate,localenddate,ordertype,orderprice,discount
0,000D3A20F23EA95811E7B7104AC23CD3,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,25
1,000D3A20F23EA95811E7B7D79FAABAA1,5,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,300
2,000D3A20F23EA95811E7B8BD71C73877,7,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,200
3,000D3A20F23EA95811E7B99F3ED09FC8,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1249,30
4,000D3A20F23EA95811E7BD373E79565E,6,2023-10-12T00:00:00.000Z,2023-10-15T23:59:00.000Z,"1,2,3",1299,20
...,...,...,...,...,...,...,...
477270,82EDCEB57826A0A811ED680C33F09E52,6,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,20
477271,82F78FC7401A9ADD11EDFA15A350D31F,5,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,200
477272,82F78FC7401AA4A811EDF992437D8CC6,5,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,200
477273,82F9662CB356B7E011EE167FC961059B,6,2023-10-30T00:00:00.000Z,2023-11-01T23:59:00.000Z,"2,3",799,20


In [16]:
# ниже экспериментальный блок с обогащением данных, в итоге не включен в итоговую таблицу(ухудшает метрику)

# попробуем обогатить данные в таблице clients_promo_october таргетом из таблицы orders
# выберем заказы за октябрь
orders_october = orders[orders['saledate'] >= '2023-10-01']
orders_october = orders_october.drop_duplicates(subset=['orderuuid'])
orders_october['clientuuid'].nunique()

# преобразуем даты в нужный формат для работы
orders_october['date'] = pd.to_datetime(orders_october['date']).dt.tz_localize(None)
clients_promo_october['localbegindate'] = pd.to_datetime(clients_promo_october['localbegindate']).dt.tz_localize(None)
clients_promo_october['localenddate'] = pd.to_datetime(clients_promo_october['localenddate']).dt.tz_localize(None)

# объединим таблицы по clientuuid
merged = pd.merge(clients_promo_october,orders_october, on='clientuuid', how='left')

# функция для проверки условий
def check_promo(row):
    # Проверяем, находится ли дата заказа внутри диапазона дат
    if row['localbegindate'] <= row['date'] <= row['localenddate']:
        # Проверяем, есть ли ordertype первой таблицы в ordertype второй таблицы
        promo_ordertype_list = map(int, row['ordertype_x'].split(','))
        if row['ordertype_y'] in promo_ordertype_list:
            return row['apply_promo']  # Возвращаем значение apply_promo
    return None

# Применяем функцию ко всем строкам
merged['apply_promo_transferred'] = merged.apply(check_promo, axis=1)

# убираем лишние столбцы и строки, возникшие при объединении
merged_clean = merged.dropna()
merged_clean = merged_clean.drop(columns=['orderuuid', 'addressid','deliverysectorid','date','saledate',
                                          'unituuid','newclient','clientordernumber','productuuid', 'categoryid', 
                                          'producttotalprice', 'menuprice', 'orderstate', 'orderpaymenttype', 
                                          'ordertotalprice', 'ordertype_y', 'apply_promo'])
merged_clean = merged_clean.rename(columns={'apply_promo_transferred': 'apply_promo',
                                            'ordertype_x': 'ordertype'})

cols = ['clientuuid', 'id', 'ordertype', 'localbegindate', 'localenddate',
       'orderprice', 'discount', 'apply_promo']  # порядок столбцов
merged_clean = merged_clean[cols]

display(merged_clean)

# скорректируем типы данных в таблицах и объединим их
# приведение типов datetime к одному формату
merged_clean['localbegindate'] = merged_clean['localbegindate'].dt.tz_localize('UTC') 
merged_clean['localenddate'] = merged_clean['localenddate'].dt.tz_localize('UTC')
merged_clean['apply_promo'] = merged_clean['apply_promo'].astype('int64')

# теперь объединяем таблицы
#train_target = pd.concat([train_target, merged_clean], ignore_index=True)

,clientuuid,id,ordertype,localbegindate,localenddate,orderprice,discount,apply_promo
1,000D3A20F23EA95811E7B7104AC23CD3,6,"1,2,3",2023-10-12,2023-10-15 23:59:00,1299,25,1.0
14,000D3A20F23EA95811E7BE56D78E3FFF,6,"1,2,3",2023-10-12,2023-10-15 23:59:00,1299,25,0.0
18,000D3A20F23EA95811E7BFD2A58E6B4F,6,"1,2,3",2023-10-12,2023-10-15 23:59:00,849,20,1.0
20,000D3A20F23EA95811E7BFEC9184B1C7,5,"1,2,3",2023-10-12,2023-10-15 23:59:00,1299,200,0.0
22,000D3A20F23EA95811E7C18DC2BD70F4,7,"1,2,3",2023-10-12,2023-10-15 23:59:00,1299,250,0.0
...,...,...,...,...,...,...,...,...
915080,6E6E02019B62AAB811ED6E4B1507945F,5,"2,3",2023-10-30,2023-11-01 23:59:00,949,150,0.0
915082,6E6E02019B62AAB811ED6E4B1507945F,5,"2,3",2023-10-30,2023-11-01 23:59:00,949,150,0.0
915140,72451D1011139F5111EE15C60E3C1F17,5,"2,3",2023-10-30,2023-11-01 23:59:00,699,150,0.0
915328,7A6C80E1D046AE3C11ED96536F40DCC7,5,"2,3",2023-10-30,2023-11-01 23:59:00,749,200,0.0


In [17]:
mobile_events

,clientuuid,visittoken,eventname,platform,timestamp
0,000D3A22FA54A81611EB315CF5443815,976627AD-76DC-41D9-981E-F2A6CA14B3A9,screen_menu,ios,2023-10-29T16:14:21.343Z
1,2A4A64AF719E994511EE3F2D8B31E188,5b22e610-c71e-4b6a-a842-71e8575ece87,screen_menu,android,2023-10-29T11:04:36.4840000Z
2,000D3AAC977BBB2F11ECDD081F5481AE,BDCEB72F-2B6A-4565-801A-6CB729279C6F,open_app,ios,2023-10-29T11:14:34.723Z
3,000D3A21DA51A81411EAE468856F96F1,A66E2DA6-4E2E-4C37-9FBF-627732623212,add_to_cart,ios,2023-10-29T14:03:08.363Z
4,000D3A23B0DC80D811E67F4D0A9CA345,1BE8139F-A3D3-439F-A51B-5CBD4D3FFF1B,screen_menu,ios,2023-10-29T08:35:40.127Z
...,...,...,...,...,...
3916649,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,open_app,ios,2023-10-14T06:00:21.558Z
3916650,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_profile,android,2023-10-13T12:55:57.7520000Z
3916651,000D3A25D54580E011E707D4762E0C84,C5EC0144-6C3C-45C2-B0A4-A7A2B6B45444,close_app,ios,2023-10-14T06:00:21.515Z
3916652,CEA31E584572ADD411ED0E4D10ED070E,5a8b6b6c-e4ad-4fd9-aad9-895fa987ddf1,screen_menu,android,2023-10-13T12:55:56.3030000Z


In [18]:
mobile = pd.DataFrame()

# создаем таблицу, где строки — уникальные 'clientuuid', а столбцы — ивенты с их количеством
event_counts = pd.crosstab(mobile_events['clientuuid'], mobile_events['eventname'])

# преобразуем результат в DataFrame
mobile = event_counts.reset_index()


In [19]:
# считаем общее количество событий для каждого clientuuid и добавляем его в таблицу mobile
event_counts_total = mobile_events.groupby('clientuuid').size().reset_index(name='totalevents')

mobile = mobile.merge(event_counts_total, on='clientuuid', how='left')

# преобразуем timestamp в формат datetime
mobile_events['timestamp'] = pd.to_datetime(mobile_events['timestamp'])

# считаем средний интервал между событиями и добавляем в таблицу mobile
mobile_events['time_diff'] = mobile_events.groupby('clientuuid')['timestamp'].diff().dt.total_seconds()
avg_time_diff = mobile_events.groupby('clientuuid')['time_diff'].mean().reset_index(name='avg_time_between_events')

mobile = mobile.merge(avg_time_diff, on='clientuuid', how='left')

# считаем активность по дням недели и добавляем в таблицу mobile
mobile_events['weekday'] = mobile_events['timestamp'].dt.strftime('%a')
weekday_activity = mobile_events.groupby(['clientuuid', 'weekday']).size().unstack().fillna(0).reset_index()

mobile = mobile.merge(weekday_activity, on='clientuuid', how='left')

mobile

,clientuuid,add_to_cart,apply_personal_offer,close_app,create_order,open_app,open_bonusaction,open_product_card,remove_from_cart,screen_cart,screen_menu,screen_profile,totalevents,avg_time_between_events,Fri,Mon,Sat,Sun,Thu,Tue,Wed
0,000D3A20F23EA95811E7B699F64188B3,12,1,45,2,43,1,14,2,13,31,5,169,-12258.231893,23.0,8.0,1.0,7.0,112.0,10.0,8.0
1,000D3A20F23EA95811E7B7104AC23CD3,23,3,24,2,25,0,3,6,9,27,5,127,-1.514873,8.0,41.0,9.0,57.0,1.0,6.0,5.0
2,000D3A20F23EA95811E7B7D79FAABAA1,4,3,11,2,10,2,4,0,3,8,14,61,8731.733333,26.0,0.0,0.0,0.0,35.0,0.0,0.0
3,000D3A20F23EA95811E7B88F5E50100B,21,0,46,4,49,0,14,7,18,41,10,210,7458.991368,8.0,44.0,0.0,17.0,29.0,62.0,50.0
4,000D3A20F23EA95811E7B8BD71C73877,8,1,33,3,34,1,7,0,7,23,7,124,-5501.271886,6.0,0.0,44.0,31.0,0.0,43.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29833,FEE611D04A50BE1A11ED390682517C35,22,5,35,4,51,3,42,14,29,114,33,352,-6201.385655,99.0,99.0,6.0,4.0,6.0,11.0,127.0
29834,FEEF35D36F3A978511EE57FC6EBC55EC,11,1,30,2,16,2,17,5,15,33,7,139,1830.181594,8.0,36.0,64.0,3.0,12.0,4.0,12.0
29835,FEEF35D36F3AA2A411EE57D13C7D60E7,24,2,31,2,32,3,44,10,37,98,33,316,-2990.979273,72.0,0.0,4.0,61.0,15.0,0.0,164.0
29836,FEEF4D137BE19C9B11ED68BADE679AA2,7,0,17,2,17,0,6,1,14,21,12,97,0.781354,0.0,0.0,0.0,24.0,0.0,73.0,0.0


In [20]:
# проверим таблицу на пропуски
mobile.isna().sum().sum()

0

In [22]:
# посчитаем долю заказов с применением промо-акции
clientuuid_bias = orders.groupby('clientuuid')['apply_promo'].mean().reset_index(name='promo_usage_rate')


In [23]:
# приведем даты к корректному типу  
orders['saledate']= pd.to_datetime(orders['saledate'])
orders['date']= pd.to_datetime(orders['date'])

# считаем среднее время между заказами для каждого клиента
time_diff = orders.groupby('clientuuid')['date'].diff().dt.days
avg_time_diff = time_diff.groupby(orders['clientuuid']).mean()
clientuuid_bias = clientuuid_bias.merge(avg_time_diff.rename('avg_time_diff'), left_on='clientuuid', right_index=True)

In [25]:
clientuuid_bias.isna().sum().sum()

0

In [27]:
train_target['avg_orderprice'] = train_target.groupby('clientuuid')['orderprice'].transform('mean')
train_target['min_orderprice'] = train_target.groupby('clientuuid')['orderprice'].transform('min')
train_target['avg_disc'] = train_target.groupby('clientuuid')['discount'].transform('mean')
train_target['max_disc'] = train_target.groupby('clientuuid')['discount'].transform('max')
train_target['orderprice_discount_interaction'] = train_target['orderprice'] * train_target['discount']

# группируем данные по клиенту
train_features = train_target.groupby('clientuuid').agg(
    max_orderprice=('orderprice', 'max'),          # считаем максимальную сумма заказа
    max_discount=('discount', 'max'),              # считаем максимальную скидку
    total_profit=('orderprice', lambda x: (x - train_target.loc[x.index, 'discount']).sum())  # считаем суммарную прибыль (после скидок)
).reset_index()

train_target = train_target.merge(train_features, on='clientuuid',how='left')

test['avg_orderprice'] = test.groupby('clientuuid')['orderprice'].transform('mean')
test['min_orderprice'] = test.groupby('clientuuid')['orderprice'].transform('min')
test['avg_disc'] = test.groupby('clientuuid')['discount'].transform('mean')
test['max_disc'] = test.groupby('clientuuid')['discount'].transform('max')
test['orderprice_discount_interaction'] = test['orderprice'] * test['discount']


# # группируем данные по клиенту и считаем признаки аналогично тренировочному датасету выше
test_features = test.groupby('clientuuid').agg(
    max_orderprice=('orderprice', 'max'),          
    max_discount=('discount', 'max'),              
    total_profit=('orderprice', lambda x: (x - test.loc[x.index, 'discount']).sum())
).reset_index()

test = test.merge(test_features, on='clientuuid',how='left')

In [29]:
# cоздаем dummy-столбцы для признаков ordertype и orderpaymenttype
orders = pd.get_dummies(orders, columns=['orderpaymenttype'], prefix='pay')
orders = pd.get_dummies(orders, columns=['ordertype'], prefix='o_type')


In [30]:
orders.head()

,orderuuid,addressid,deliverysectorid,clientuuid,date,saledate,unituuid,newclient,clientordernumber,productuuid,categoryid,producttotalprice,menuprice,orderstate,ordertotalprice,apply_promo,pay_0,pay_1,pay_2,o_type_1,o_type_2,o_type_3
0,020DD76CCD949AE011EDE031ED5D76FD,0.0,0.0,000D3AAC9DCABB2E11EBE0614C89D044,2023-04-21,2023-04-21 20:51:26+00:00,000D3A2480C380DA11E692CFB9F92484,0,18,000D3A240C71BE9A11E719BE2AB264A6,4,1.0,45.0,4,1.0,0,False,False,True,False,False,True
1,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02 21:31:32+00:00,000D3A26B5B080DF11E745FD0474B95C,0,29,11EC9AABAD354BE051CCF1EE1F83A8D0,1,485.0,485.0,4,1164.0,0,False,False,True,True,False,False
2,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02 21:31:32+00:00,000D3A26B5B080DF11E745FD0474B95C,0,29,11EB846583CB3BDEA80D1A7F2E184D40,1,589.0,589.0,4,1164.0,0,False,False,True,True,False,False
3,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02 21:31:32+00:00,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB267FD,4,45.0,45.0,4,1164.0,0,False,False,True,True,False,False
4,02164D9B0F67BC3311EE610987AF48C7,0.0,0.0,000D3A21DA51A81411EAF10674007D74,2023-10-02,2023-10-02 21:31:32+00:00,000D3A26B5B080DF11E745FD0474B95C,0,29,000D3A240C71BE9A11E719BE2AB264A6,4,45.0,45.0,4,1164.0,0,False,False,True,True,False,False


In [34]:
# считаем еще одиннаборм признаков
grouped_1 = orders.groupby('clientuuid').agg(
    producttotalprice_sum=('producttotalprice', 'sum'),
    producttotalprice_mean=('producttotalprice', 'mean'),
    menuprice_sum=('menuprice', 'sum'),
    menuprice_mean=('menuprice', 'mean'),
    clientordernumber_max=('clientordernumber', 'max'),
    clientordernumber_min=('clientordernumber', 'min'),
    clientordernumber_avg=('clientordernumber', 'mean'),
    orderstate_mean=('orderstate', 'mean'),
    pay_0_count=('pay_0', 'sum'),
    pay_1_count=('pay_1', 'sum'),
    pay_2_count=('pay_2', 'sum'),
    o_type_1_count=('o_type_1', 'sum'),
    o_type_2_count=('o_type_2', 'sum'),
    o_type_3_count=('o_type_3', 'sum'),
    ordertotalprice_mean=('ordertotalprice', 'mean'),
    ordertotalprice_std=('ordertotalprice', 'std'),
    ordertotalprice_sum=('ordertotalprice', 'sum'),
    apply_promo_sum=('apply_promo', 'sum'),
    unique_productuuid=('productuuid', 'nunique'),
    unique_categoryid=('categoryid', 'nunique')
).reset_index()

clientuuid_bias = clientuuid_bias.merge(grouped_1, on='clientuuid', how='left')
clientuuid_bias.head()

,clientuuid,promo_usage_rate,avg_time_diff,producttotalprice_sum,producttotalprice_mean,menuprice_sum,menuprice_mean,clientordernumber_max,clientordernumber_min,clientordernumber_avg,orderstate_mean,pay_0_count,pay_1_count,pay_2_count,o_type_1_count,o_type_2_count,o_type_3_count,ordertotalprice_mean,ordertotalprice_std,ordertotalprice_sum,apply_promo_sum,unique_productuuid,unique_categoryid
0,000D3A20F23EA95811E7B7104AC23CD3,0.837838,-0.589041,11081.0,149.743243,14889.0,201.202703,151,142,146.918919,4.000000,74,0,0,74,0,0,1139.810811,230.190432,84346.0,62,21,5
1,000D3A20F23EA95811E7B88F5E50100B,0.115385,-0.627451,9025.0,173.557692,10040.0,193.076923,66,58,62.096154,4.000000,0,0,52,52,0,0,1150.288462,486.131965,59815.0,6,17,6
2,000D3A20F23EA95811E7B8BD71C73877,0.142857,-0.588235,8575.0,245.000000,10584.0,302.400000,121,112,116.942857,4.000000,0,0,35,35,0,0,945.600000,408.621657,33096.0,5,25,6
3,000D3A20F23EA95811E7B95FE7E6321C,0.104167,0.702128,8353.0,174.020833,10460.0,217.916667,96,86,90.020833,4.729167,11,0,37,47,0,1,904.229167,243.612025,43403.0,5,25,6
4,000D3A20F23EA95811E7B969B5BD2C3F,0.000000,-1.454545,11226.0,330.176471,13449.0,395.558824,116,107,111.205882,4.000000,0,0,34,21,0,13,1170.529412,443.212873,39798.0,0,26,6


In [35]:
# собираем все тренировочные данные в одну таблицу
data_full = train_target.merge(mobile, on='clientuuid', how='left')
data_full = data_full.merge(clientuuid_bias, on='clientuuid', how='left')


In [36]:
data_full = data_full.drop(['clientuuid','localbegindate', 'localenddate'],axis=1)

# заполним строковые пропуски текстом 'missing'
data_full.loc[:, data_full.select_dtypes(include=['object']).columns] = data_full.select_dtypes(include=['object']).fillna('missing')

# заполним числовые пропуски значением 0
data_full.loc[:, data_full.select_dtypes(include=['number']).columns] = data_full.select_dtypes(include=['number']).fillna(0)

data_full.isna().sum().sum()

0

In [37]:

data_full.head(10)

,id,ordertype,orderprice,discount,apply_promo,avg_orderprice,min_orderprice,avg_disc,max_disc,orderprice_discount_interaction,max_orderprice,max_discount,total_profit,add_to_cart,apply_personal_offer,close_app,create_order,open_app,open_bonusaction,open_product_card,remove_from_cart,screen_cart,screen_menu,screen_profile,totalevents,avg_time_between_events,Fri,Mon,Sat,Sun,Thu,Tue,Wed,promo_usage_rate,avg_time_diff,producttotalprice_sum,producttotalprice_mean,menuprice_sum,menuprice_mean,clientordernumber_max,clientordernumber_min,clientordernumber_avg,orderstate_mean,pay_0_count,pay_1_count,pay_2_count,o_type_1_count,o_type_2_count,o_type_3_count,ordertotalprice_mean,ordertotalprice_std,ordertotalprice_sum,apply_promo_sum,unique_productuuid,unique_categoryid
0,7,"2,3",699,200,0,974.0,699,112.5,200,139800,1249,200,1723,11.0,0.0,43.0,4.0,43.0,0.0,22.0,2.0,8.0,54.0,0.0,187.0,-26.726290,54.0,63.0,3.0,12.0,6.0,17.0,32.0,0.000000,-1.093750,9179.0,278.151515,12200.0,369.696970,118.0,97.0,105.484848,4.000000,0.0,0.0,33.0,20.0,0.0,13.0,1853.727273,2150.551765,61173.0,0.0,24.0,5.0
1,6,"1,2,3",1249,25,0,974.0,699,112.5,200,31225,1249,200,1723,11.0,0.0,43.0,4.0,43.0,0.0,22.0,2.0,8.0,54.0,0.0,187.0,-26.726290,54.0,63.0,3.0,12.0,6.0,17.0,32.0,0.000000,-1.093750,9179.0,278.151515,12200.0,369.696970,118.0,97.0,105.484848,4.000000,0.0,0.0,33.0,20.0,0.0,13.0,1853.727273,2150.551765,61173.0,0.0,24.0,5.0
2,5,"2,3",799,200,0,1024.0,799,225.0,250,159800,1249,250,1598,5.0,0.0,16.0,2.0,15.0,0.0,6.0,0.0,11.0,12.0,30.0,97.0,-1.075625,8.0,0.0,0.0,79.0,0.0,10.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3,5,"1,2,3",1249,250,0,1024.0,799,225.0,250,312250,1249,250,1598,5.0,0.0,16.0,2.0,15.0,0.0,6.0,0.0,11.0,12.0,30.0,97.0,-1.075625,8.0,0.0,0.0,79.0,0.0,10.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
4,6,"2,3",699,30,0,974.0,699,140.0,250,20970,1249,250,1668,9.0,0.0,19.0,3.0,21.0,0.0,12.0,1.0,11.0,16.0,11.0,103.0,-9245.337941,28.0,47.0,0.0,0.0,24.0,4.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
5,5,"1,2,3",1249,250,0,974.0,699,140.0,250,312250,1249,250,1668,9.0,0.0,19.0,3.0,21.0,0.0,12.0,1.0,11.0,16.0,11.0,103.0,-9245.337941,28.0,47.0,0.0,0.0,24.0,4.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
6,6,"2,3",799,20,0,1024.0,799,135.0,250,15980,1249,250,1778,6.0,1.0,22.0,2.0,23.0,0.0,11.0,0.0,9.0,23.0,1.0,98.0,-76.674598,0.0,0.0,0.0,49.0,10.0,39.0,0.0,0.210526,-6.722222,6439.0,338.894737,7210.0,379.473684,32.0,24.0,28.789474,4.000000,0.0,0.0,19.0,18.0,0.0,1.0,871.000000,210.884645,16549.0,4.0,13.0,5.0
7,7,"1,2,3",1249,250,0,1024.0,799,135.0,250,312250,1249,250,1778,6.0,1.0,22.0,2.0,23.0,0.0,11.0,0.0,9.0,23.0,1.0,98.0,-76.674598,0.0,0.0,0.0,49.0,10.0,39.0,0.0,0.210526,-6.722222,6439.0,338.894737,7210.0,379.473684,32.0,24.0,28.789474,4.000000,0.0,0.0,19.0,18.0,0.0,1.0,871.000000,210.884645,16549.0,4.0,13.0,5.0
8,6,"2,3",799,20,0,899.0,799,85.0,150,15980,999,150,1628,19.0,3.0,31.0,2.0,30.0,3.0,4.0,3.0,23.0,29.0,20.0,167.0,8342.383849,0.0,66.0,101.0,0.0,0.0,0.0,0.0,0.795181,0.341463,6810.0,82.048193,10594.0,127.638554,137.0,127.0,132.554217,5.686747,0.0,0.0,83.0,76.0,0.0,7.0,747.710843,252.580995,62060.0,66.0,25.0,6.0
9,7,"1,2,3",999,150,0,899.0,799,85.0,150,149850,999,150,1628,19.0,3.0,31.0,2.0,30.0,3.0,4.0,3.0,23.0,29.0,20.0,167.0,8342.383849,0.0,66.0,101.0,0.0,0.0,0.0,0.0,0.795181,0.341463,6810.0,82.048193,10594.0,127.638554,137.0,127.0,132.554217,5.686747,0.0,0.0,83.0,76.0,0.0,7.0,747.710843,252.580995,62060.0,66.0,25.0,6.0


In [38]:
# задаем константы
RANDOM_STATE = 1
TEST_SIZE = 0.1

In [39]:
# сформируем наборы данных для обучения и тестов
X_train, X_test, y_train, y_test = train_test_split(
    data_full.drop(['apply_promo'], axis=1),
    data_full['apply_promo'],
    test_size = TEST_SIZE,
    stratify=data_full['apply_promo'],
    random_state = RANDOM_STATE)

In [40]:
%%time

# итоговый пайплайн для CatBoost не формируем: 
# подготовка данных не требуется - подаем исходные наборы X_train, y_train и модель одна

estimator = CatBoostClassifier(verbose=200)
cat_features = ['id','ordertype']

# Приведение категориальных столбцов к строковому типу
for col in cat_features:
    X_train[col] = X_train[col].astype(str)

# задаем пока набор гиперпараметров
param_grid_4 = [
    {
        'iterations': [2000],
        'thread_count': [-1],
        'random_seed': [1],
        'eta': [0.01],
        'l2_leaf_reg': [3],
        'min_data_in_leaf': [10],
        'colsample_bylevel': [0.7],
        'max_bin': [20],
        'subsample': [0.9],
        'depth': [5]
    }
]

# перебор гиперпараметров с помощью GridSearchCV
grid_search_4 = GridSearchCV(
    estimator, 
    param_grid_4, 
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)

# обучим модель
model_4 = grid_search_4.fit(X_train, y_train,cat_features=cat_features)

# лучший показатель roc_auc
best_score_roc_auc_4 = model_4.best_score_

# лучшие параметры модели
best_params_4 = model_4.best_params_

# выводим результат
print(f'Лучший roc_auc: {round(best_score_roc_auc_4, 3)}')
print(f'Лучшие параметры модели: {best_params_4}')

0:	learn: 0.6762946	total: 92.2ms	remaining: 3m 4s
0:	learn: 0.6762478	total: 94.6ms	remaining: 3m 9s
0:	learn: 0.6763298	total: 94.9ms	remaining: 3m 9s
0:	learn: 0.6762856	total: 92.6ms	remaining: 3m 5s
0:	learn: 0.6763688	total: 104ms	remaining: 3m 27s
200:	learn: 0.1218682	total: 10.8s	remaining: 1m 36s
200:	learn: 0.1201081	total: 10.9s	remaining: 1m 37s
200:	learn: 0.1223692	total: 10.9s	remaining: 1m 37s
200:	learn: 0.1222121	total: 11.1s	remaining: 1m 38s
200:	learn: 0.1215235	total: 11.1s	remaining: 1m 39s
400:	learn: 0.1107129	total: 21.2s	remaining: 1m 24s
400:	learn: 0.1079760	total: 21.3s	remaining: 1m 24s
400:	learn: 0.1095368	total: 21.4s	remaining: 1m 25s
400:	learn: 0.1099654	total: 21.6s	remaining: 1m 26s
400:	learn: 0.1104074	total: 21.7s	remaining: 1m 26s
600:	learn: 0.1019020	total: 34.9s	remaining: 1m 21s
600:	learn: 0.1046806	total: 35.1s	remaining: 1m 21s
600:	learn: 0.1041200	total: 35.2s	remaining: 1m 21s
600:	learn: 0.1033556	total: 35.7s	remaining: 1m 23s
600

In [41]:
# получаем важность признаков
feature_importances = model_4.best_estimator_.get_feature_importance()
feature_names = X_train.columns

# создаем DataFrame с признаками и их важностью
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
})

# сортируем по важности
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# установим максимальное количество выводимых строк на None (для отображения всех строк)
pd.set_option('display.max_rows', None)

# выводим отсортированную таблицу
display(feature_importance_df)

# вернем настройку отображения строк к значению по умолчанию
pd.reset_option('display.max_rows')

,Feature,Importance
32,promo_usage_rate,7.895899
13,apply_personal_offer,6.691908
51,apply_promo_sum,5.382862
11,total_profit,4.781878
24,avg_time_between_events,3.120114
17,open_bonusaction,2.687415
14,close_app,2.601033
9,max_orderprice,2.470992
5,min_orderprice,2.445460
34,producttotalprice_sum,2.437127


In [42]:
# приведение категориальных столбцов к строковому типу
for col in cat_features:
    X_test[col] = X_test[col].astype(str)


# предсказание вероятностей для тестовой выборки
y_test_proba_4 = model_4.predict_proba(X_test)[:, 1]

# рассчитываем метрику ROC_AUC на тестовой выборке
roc_auc_test_4 = roc_auc_score(y_test, y_test_proba_4)
print(f'Метрика ROC_AUC на тестовой выборке: {round(roc_auc_test_4, 3)}')

Метрика ROC_AUC на тестовой выборке: 0.855


In [43]:
# собираем все тестовые данные в одну таблицу
test_full = test.merge(mobile, on='clientuuid', how='left')
test_full = test_full.merge(clientuuid_bias, on='clientuuid', how='left')

# убираем ненужные столбцы аналогично тренировочному набору
test_full = test_full.drop(['clientuuid','localbegindate', 'localenddate'],axis=1)

test_full.head()

,id,ordertype,orderprice,discount,avg_orderprice,min_orderprice,avg_disc,max_disc,orderprice_discount_interaction,max_orderprice,max_discount,total_profit,add_to_cart,apply_personal_offer,close_app,create_order,open_app,open_bonusaction,open_product_card,remove_from_cart,screen_cart,screen_menu,screen_profile,totalevents,avg_time_between_events,Fri,Mon,Sat,Sun,Thu,Tue,Wed,promo_usage_rate,avg_time_diff,producttotalprice_sum,producttotalprice_mean,menuprice_sum,menuprice_mean,clientordernumber_max,clientordernumber_min,clientordernumber_avg,orderstate_mean,pay_0_count,pay_1_count,pay_2_count,o_type_1_count,o_type_2_count,o_type_3_count,ordertotalprice_mean,ordertotalprice_std,ordertotalprice_sum,apply_promo_sum,unique_productuuid,unique_categoryid
0,7,"2,3",799,200,799.0,799,200.0,200,159800,799,200,599,9.0,2.0,16.0,2.0,16.0,3.0,4.0,2.0,9.0,30.0,14.0,107.0,7523.623462,0.0,14.0,57.0,33.0,3.0,0.0,0.0,0.066667,-0.241379,5659.0,188.633333,6554.0,218.466667,49.0,39.0,44.366667,4.0,8.0,0.0,22.0,26.0,0.0,4.0,569.033333,113.087541,17071.0,2.0,12.0,4.0
1,5,"2,3",699,200,699.0,699,200.0,200,139800,699,200,499,27.0,3.0,38.0,2.0,39.0,3.0,17.0,17.0,36.0,71.0,14.0,267.0,-7183.878981,12.0,14.0,48.0,61.0,124.0,8.0,0.0,0.465517,0.543860,10072.0,173.655172,12591.0,217.086207,102.0,88.0,95.775862,4.0,0.0,0.0,58.0,58.0,0.0,0.0,1091.827586,566.374067,63326.0,27.0,26.0,6.0
2,6,"2,3",799,20,799.0,799,20.0,20,15980,799,20,779,7.0,1.0,23.0,2.0,23.0,1.0,2.0,2.0,10.0,14.0,3.0,88.0,-7532.489609,59.0,0.0,0.0,0.0,29.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,"2,3",799,200,1024.0,799,200.0,200,159800,1249,200,1648,18.0,2.0,14.0,2.0,15.0,2.0,19.0,7.0,11.0,34.0,4.0,128.0,3334.263756,83.0,0.0,1.0,41.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,"2,3",799,200,799.0,799,200.0,200,159800,799,200,599,12.0,2.0,9.0,3.0,9.0,0.0,8.0,0.0,6.0,17.0,8.0,74.0,-21067.224904,35.0,0.0,34.0,0.0,0.0,5.0,0.0,0.379310,-4.285714,7216.0,248.827586,8323.0,287.000000,23.0,14.0,19.689655,4.0,0.0,0.0,29.0,0.0,0.0,29.0,865.206897,387.308513,25091.0,11.0,15.0,5.0


In [44]:
# смотрим на итоговую тесовую таблицу и количество пропусков
test_full.info()

test_full.isna().sum().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6806 entries, 0 to 6805
Data columns (total 54 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               6806 non-null   int64  
 1   ordertype                        6806 non-null   object 
 2   orderprice                       6806 non-null   int64  
 3   discount                         6806 non-null   int64  
 4   avg_orderprice                   6806 non-null   float64
 5   min_orderprice                   6806 non-null   int64  
 6   avg_disc                         6806 non-null   float64
 7   max_disc                         6806 non-null   int64  
 8   orderprice_discount_interaction  6806 non-null   int64  
 9   max_orderprice                   6806 non-null   int64  
 10  max_discount                     6806 non-null   int64  
 11  total_profit                     6806 non-null   int64  
 12  add_to_cart         

68594

In [45]:
# приведение категориальных столбцов к строковому типу
for col in cat_features:
    test_full[col] = test_full[col].astype(str)

# заполним строковые пропуски текстом missing
test_full.loc[:, test_full.select_dtypes(include=['object']).columns] = test_full.select_dtypes(include=['object']).fillna('missing')

# заполненим числовые пропуски значением 0
test_full.loc[:, test_full.select_dtypes(include=['number']).columns] = test_full.select_dtypes(include=['number']).fillna(0)

test_full.isna().sum().sum()

0

In [46]:
# генерируем прогнозы на тестовой выборке
test_full['apply_promo'] = model_4.best_estimator_.predict_proba(test_full)[:, 1]
test_full.head()

,id,ordertype,orderprice,discount,avg_orderprice,min_orderprice,avg_disc,max_disc,orderprice_discount_interaction,max_orderprice,max_discount,total_profit,add_to_cart,apply_personal_offer,close_app,create_order,open_app,open_bonusaction,open_product_card,remove_from_cart,screen_cart,screen_menu,screen_profile,totalevents,avg_time_between_events,Fri,Mon,Sat,Sun,Thu,Tue,Wed,promo_usage_rate,avg_time_diff,producttotalprice_sum,producttotalprice_mean,menuprice_sum,menuprice_mean,clientordernumber_max,clientordernumber_min,clientordernumber_avg,orderstate_mean,pay_0_count,pay_1_count,pay_2_count,o_type_1_count,o_type_2_count,o_type_3_count,ordertotalprice_mean,ordertotalprice_std,ordertotalprice_sum,apply_promo_sum,unique_productuuid,unique_categoryid,apply_promo
0,7,"2,3",799,200,799.0,799,200.0,200,159800,799,200,599,9.0,2.0,16.0,2.0,16.0,3.0,4.0,2.0,9.0,30.0,14.0,107.0,7523.623462,0.0,14.0,57.0,33.0,3.0,0.0,0.0,0.066667,-0.241379,5659.0,188.633333,6554.0,218.466667,49.0,39.0,44.366667,4.0,8.0,0.0,22.0,26.0,0.0,4.0,569.033333,113.087541,17071.0,2.0,12.0,4.0,0.010904
1,5,"2,3",699,200,699.0,699,200.0,200,139800,699,200,499,27.0,3.0,38.0,2.0,39.0,3.0,17.0,17.0,36.0,71.0,14.0,267.0,-7183.878981,12.0,14.0,48.0,61.0,124.0,8.0,0.0,0.465517,0.543860,10072.0,173.655172,12591.0,217.086207,102.0,88.0,95.775862,4.0,0.0,0.0,58.0,58.0,0.0,0.0,1091.827586,566.374067,63326.0,27.0,26.0,6.0,0.047592
2,6,"2,3",799,20,799.0,799,20.0,20,15980,799,20,779,7.0,1.0,23.0,2.0,23.0,1.0,2.0,2.0,10.0,14.0,3.0,88.0,-7532.489609,59.0,0.0,0.0,0.0,29.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.014127
3,5,"2,3",799,200,1024.0,799,200.0,200,159800,1249,200,1648,18.0,2.0,14.0,2.0,15.0,2.0,19.0,7.0,11.0,34.0,4.0,128.0,3334.263756,83.0,0.0,1.0,41.0,0.0,3.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.035481
4,7,"2,3",799,200,799.0,799,200.0,200,159800,799,200,599,12.0,2.0,9.0,3.0,9.0,0.0,8.0,0.0,6.0,17.0,8.0,74.0,-21067.224904,35.0,0.0,34.0,0.0,0.0,5.0,0.0,0.379310,-4.285714,7216.0,248.827586,8323.0,287.000000,23.0,14.0,19.689655,4.0,0.0,0.0,29.0,0.0,0.0,29.0,865.206897,387.308513,25091.0,11.0,15.0,5.0,0.028670


In [47]:
# записываем прогноз в файл
test_full['apply_promo'].to_csv('submit_34.csv')